In [1]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import keras
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
url = '/content/drive/MyDrive/dataset/test_set/test_set'

In [3]:
base_model = InceptionV3(input_shape=(256, 256, 3), include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
for layer in base_model.layers:
    layer.trainable = False

In [5]:
X = Flatten()(base_model.output)
X = Dense(units=2, activation='sigmoid')(X)

model = Model(base_model.input, X)
model.compile(
    optimizer='adam',
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

#model.summary()

# ***PREPROCESSING*** DATA WITH IMAGE GENERATOR

In [6]:
CATEGORIES = ['cats', 'dogs']
train_dataset = []
for category in CATEGORIES:
  path = f'{url}/{category}'
  class_num = CATEGORIES.index(category)
  for img in os.listdir(path):
    try:
      image = cv2.imread(f'{path}/{img}')
      image = cv2.resize(image, (256, 256))
      train_dataset.append([image, class_num])
    except:
      print('Data not added')

Data not added


In [7]:
X = []
Y = []
random.shuffle(train_dataset)
for feat, label in train_dataset:
    X.append(feat)
    Y.append(label)

In [8]:
mc = ModelCheckpoint(filepath='./base_model.keras',
                     monitor='val_accuracy',
                     verbose=1,
                     save_best_only=True)

es = EarlyStopping(monitor='val_accuracy',
                   min_delta=0.05,
                   patience=5,
                   verbose=1)

cb = [mc, es]

In [9]:
X = np.array(X).reshape(-1, 256, 256, 3)
X = X.astype('float32')
X /= 255

Y = keras.utils.to_categorical(Y, 2)

In [10]:
his = model.fit(X, Y, batch_size=10, verbose=1, callbacks=cb)

157/157 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.9311 - loss: 0.7260


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [11]:
model.save('/content/drive/MyDrive/model.h5')

TESTING THE MODEL NOW

In [13]:
image = cv2.imread('/content/drive/MyDrive/dataset/training_set/training_set/cats/cat.1.jpg')
image = cv2.resize(image, (256, 256))

image = image.reshape(-1, 256, 256, 3)
image = image.astype('float32')
image /= 255

In [14]:

result = model.predict(image)
print(result.all())
if result.any() == 0:
    print('cat')
else:
    print('dog')

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
True
dog
